In [ ]:
### НУЖНЫЕ ИМПОРТЫ
# !pip install python-dotenv==1.0.1 mistralai

In [1]:
from dotenv import load_dotenv
from IPython.display import display, Markdown
import re
import os
from mistralai import Mistral

## 1. USER QUERY

In [ ]:
# Пока вопрос в виде строки, потом видимо хотим сделать как .py скрипт
user_query = "Как заселить НИП"

## 2. RETRIEVAL
#### 2.1 BUILDING QUERY EMBEDDINGS

#### 2.2 CHANKING DATA

наверное нужно все данные заранее сделать, а не делать это все в онлайне, просто BERT офигеет столько данных через себя прогонять

#### 2.3 SEARCH

## 3. PROMPT BUILDING

## 4. LLM ANSWER

In [1]:
from dotenv import load_dotenv
import os

load_dotenv('.env')
MISTRAL_API_KEY = os.getenv('API_KEY')

In [2]:
from mistralai import Mistral
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
vectorstore = Chroma(persist_directory="./terraria_db", embedding_function=embeddings)
mistral = Mistral(api_key=MISTRAL_API_KEY)

d:\дз\TerrariaRAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\nvclon\AppData\Local\Temp\ipykernel_26732\3018973633.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="./terraria_db", embedding_function=embeddings)


In [3]:
models = mistral.models.list()  # Проверка что ключ рабочий
print([(m.id, m.description) for m in models.data])

[('mistral-medium-2505', 'Our frontier-class multimodal model released May 2025.'), ('mistral-large-latest', 'Official mistral-large-latest Mistral AI model'), ('mistral-medium-2508', 'Update on Mistral Medium 3 with improved capabilities.'), ('mistral-medium-latest', 'Update on Mistral Medium 3 with improved capabilities.'), ('mistral-medium', 'Update on Mistral Medium 3 with improved capabilities.'), ('ministral-3b-2410', "World's best edge model."), ('ministral-3b-latest', "World's best edge model."), ('ministral-8b-2410', 'Powerful edge model with extremely high performance/price ratio.'), ('ministral-8b-latest', 'Powerful edge model with extremely high performance/price ratio.'), ('open-mistral-7b', 'Our first dense model released September 2023.'), ('mistral-tiny', 'Our first dense model released September 2023.'), ('mistral-tiny-2312', 'Our first dense model released September 2023.'), ('open-mistral-nemo', 'Our best multilingual open source model released July 2024.'), ('open-m

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

SYSTEM_PROMPT = """Ты — TerrariaExpert, интеллектуальный ассистент по игре *Terraria*.

🔹 Твоя цель — давать точные, подробные и проверенные ответы о механиках, предметах, рецептах, врагах, биомах, версиях и событиях Terraria.  
🔹 Ты работаешь в связке с системой Retrieval-Augmented Generation (RAG), и получаешь контекст из векторной базы данных Chroma, в которой сохранены статьи и данные из Terraria Wiki и других авторитетных источников.  

## 📘 Основные принципы:
1. **Опора на факты.**  
   Отвечай строго на основе предоставленного контекста.  
   Если информации недостаточно — прямо скажи:  
   > "Я не нашёл точной информации об этом в базе знаний."  
   и кратко объясни, чего именно не хватает.

2. **Без домыслов и фантазий.**  
   Не придумывай данных, не выдумывай крафты, эффекты, характеристики и т.д.  
   Все утверждения должны быть подтверждены контекстом или официальной игровой логикой Terraria.

3. **Язык ответа.**  
   Отвечай на **том же языке**, на котором задан вопрос (русский или английский).  
   Используй правильные игровые термины и переводы (например, "Меч из пепельного дерева", "Lihzahrd Furnace").

4. **Формат ответа.**  
   - Используй Markdown для структурирования.  
   - Если запрос технический (например, "покажи рецепт предмета"), выдай результат в структурированном виде:
     ```
     🔨 **Рецепт: Лихзардовая печь**
     - Станок: Furnace
     - Компоненты:
       • 20 Lihzahrd Brick
       • 5 Iron Bar
     ```
   - Если запрос концептуальный (например, "как работает экспертный режим?"), пиши сжатое, но понятное объяснение.

5. **Множественные результаты.**  
   Если контекст возвращает несколько совпадений, перечисли их и кратко опиши каждый.

6. **Актуальность и версии.**  
   Если в данных указана версия (например, "Desktop 1.4.4" или "1.3.5.3"), обязательно упомяни это в ответе, чтобы различать различия между изданиями Terraria.

7. **Формулировка ответов.**  
   - Будь точен, ясен, без избыточной воды.  
   - Используй списки, таблицы и выделение, чтобы ответ было удобно читать.  
   - Если запрос связан с механиками, объясняй *пошагово*, как они работают.

## ⚙️ Примеры поведения:
- ❌ **Плохо:** “Я думаю, этот предмет крафтится из металла и дерева.”  
- ✅ **Хорошо:** “Согласно данным Terraria Wiki, предмет крафтится на Iron Anvil из 10 Iron Bar и 2 Wood.”

- ❌ **Плохо:** “Возможно, этот NPC появляется ночью.”  
- ✅ **Хорошо:** “В контексте указано, что этот NPC появляется ночью при выполнении условий X, Y и Z.”

## 🧠 Если контекста нет:
Если RAG не вернул данных или текст не содержит точного совпадения, скажи:
> “У меня нет данных об этом в текущей базе Terraria. Возможно, информация отсутствует или относится к модам, не включённым в базу.”

---

Ты должен действовать как **эксперт-энциклопедия Terraria**,  
но сдержанный и точный, не выходящий за рамки контекста и базы знаний.

"""

USER_PROMPT = """Контекст:
{context}

Вопрос: {question}
"""

def rag_ask(question: str):
    docs = retriever._get_relevant_documents(question + ' Рецепт' if 'рецепт' in question.lower() else '', run_manager=None)
    context = "\n\n".join([d.page_content for d in docs])

    # Отправляем в Mistral
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT.format(context=context, question=question)},
    ]

    response = mistral.chat.complete(
        model="open-mistral-nemo",
        messages=messages,
        temperature=0.2
    )

    return response.choices[0].message.content, docs

if __name__ == "__main__":
   question = "Сколько всего кроватей в террария"
   answer, docs = rag_ask(question)
   print("\n💬 Вопрос:", question)
   print("🧠 Ответ:", answer)
   print("\n📚 Источники:")
   for doc in docs:
        print(f"- {doc}")


💬 Вопрос: Сколько всего кроватей в террария
🧠 Ответ: В Terraria есть несколько типов кроватей, которые можно использовать для спавна в мире. Вот список всех типов кроватей в Terraria:

* Кровать из пепельного дерева
* Кровать из садового дерева
* Кровать из железного дерева
* Кровать из мрачного дерева
* Кровать из пальмового дерева
* Кровать из розового дерева
* Кровать из тикового дерева
* Кровать из тусклого дерева
* Кровать из вершины мира

Итого, в Terraria 9 разных типов кроватей.

📚 Источники:
- page_content='|-
| align=center | 
| 
| 
| 
| align=center | 
|-
| align=center | 
| 
| 
| 
| align=center | 
|-
| align=center | 
| 
| 
| 
| align=center | 
|-
| align=center | 
| 
| 
| 
| align=center | 
|-
| align=center | 
| 
| 
| 
| align=center | 
|-
| align=center | 
| 
| 
| 
| align=center | 
|-
| align=center | 
| 
| 
| 
| align=center | 
|-
| align=center | 
| 
| 
| + 
| align=center | 
|-
| align=center | 
| 
| 
| 
| align=center | 
|-
| align=center | 
| 
| 
| 
| align=cente